<a href="https://colab.research.google.com/github/adilsonalbino/SQL-e-PYSPARK/blob/main/14_Fun%C3%A7%C3%B5es_de_agrega%C3%A7%C3%A3o_M%C3%81XIMO_M%C3%8DNIMO_SOMA%2C_M%C3%89DIA_TOTAL_no_SQL_e_PYSPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#14-Funções de agregação no SQL e PYSPARK
by Adilson Albino

In [8]:
#instalando pyspark
!pip install pyspark

#importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Criando app spark
spark = SparkSession.builder \
.appName("Spark")\
.getOrCreate()

In [2]:
#Variavel com a informação do caminho do dataset
file_path_carros = "/content/drive/MyDrive/Colab Notebooks/PySpark_SQL/DATASET_CARROS/modelo_carro.csv"

In [10]:
#Carregando o dataset
df_carros = spark\
.read.format("csv")\
.option("header", True)\
.load(file_path_carros)\

df_carros = df_carros.withColumn(
    "preco",
    regexp_replace("preco", "\$", "")
    .cast(DoubleType())
)


df_carros.show(10)

+--------+------------+--------+---------+
|id_carro|modelo_carro|   preco|cod_marca|
+--------+------------+--------+---------+
|       1|      Avalon|78401.95|       54|
|       2|         RDX|95987.38|        1|
|       3|        Golf|61274.55|       55|
|       4|          EX|84981.12|       23|
|       5|      Escort|77466.89|       17|
|       6|  Expedition|84698.71|       17|
|       7|     Voyager|95567.75|       42|
|       8|       Civic|84749.22|       20|
|       9|    Defender|98600.79|       29|
|      10|V8 Vantage S|94791.61|        2|
+--------+------------+--------+---------+
only showing top 10 rows



#Utilizando SQL

In [11]:
#Criando tabela temporária
df_carros_sql = df_carros.createOrReplaceTempView("carros")

In [42]:
#Utilizando MAX, MIN, SUM, AVG, COUNT
df_carros_sql = spark.sql("""
  SELECT

      MAX(preco) AS MAIOR,
      MIN(preco) AS MENOR,
      SUM(preco) AS SOMA,
      AVG(preco) AS MEDIA,
      COUNT(*) AS TOTAL_CARROS

  FROM carros
""")
df_carros_sql.show()

+--------+--------+-------------------+-----------------+------------+
|   MAIOR|   MENOR|               SOMA|            MEDIA|TOTAL_CARROS|
+--------+--------+-------------------+-----------------+------------+
|99938.89|45082.42|7.266464454000008E7|72664.64454000008|        1000|
+--------+--------+-------------------+-----------------+------------+



In [44]:
#Fazendo agrupamento (GROUP BY)
df_carros_sql = spark.sql("""
  SELECT
      modelo_carro,
      MAX(preco) AS MAIOR,
      MIN(preco) AS MENOR,
      SUM(preco) AS SOMA,
      AVG(preco) AS MEDIA,
      COUNT(*) AS TOTAL_CARROS
  FROM carros
  GROUP BY modelo_carro
""")
df_carros_sql.show()

+------------+--------+--------+------------------+------------------+------------+
|modelo_carro|   MAIOR|   MENOR|              SOMA|             MEDIA|TOTAL_CARROS|
+------------+--------+--------+------------------+------------------+------------+
| Savana 1500|85392.12|49351.82|         324443.83|         64888.766|           5|
|      Cirrus|81784.17|81784.17|          81784.17|          81784.17|           1|
|         944|87930.76|87930.76|          87930.76|          87930.76|           1|
|      Avanti|98629.28|98629.28|          98629.28|          98629.28|           1|
|     Outlook|68318.15|49510.92|117829.06999999999|58914.534999999996|           2|
|     Contour|72937.19|72937.19|          72937.19|          72937.19|           1|
|         S60|85793.33| 48719.1|         377417.08| 62902.84666666667|           6|
|        1500|70293.96|45539.22|115833.18000000001|57916.590000000004|           2|
|       XLR-V|75537.75|50360.58|         125898.33|         62949.165|      

In [45]:
#Ordenando com ORDER BY
df_carros_sql = spark.sql("""

  SELECT
      modelo_carro,
      MAX(preco) AS MAIOR,
      MIN(preco) AS MENOR,
      SUM(preco) AS SOMA,
      AVG(preco) AS MEDIA,
      COUNT(*) AS TOTAL_CARROS
  FROM carros
  GROUP BY modelo_carro
  ORDER by MAIOR desc
""")
df_carros_sql.show()

+-----------------+--------+--------+------------------+-----------------+------------+
|     modelo_carro|   MAIOR|   MENOR|              SOMA|            MEDIA|TOTAL_CARROS|
+-----------------+--------+--------+------------------+-----------------+------------+
|           Carens|99938.89|99938.89|          99938.89|         99938.89|           1|
|               X5|99866.05|93976.05|         289115.56|96371.85333333333|           3|
|     Express 2500|99803.08|56653.51|         156456.59|        78228.295|           2|
|           Sephia|99726.07|96908.44|         196634.51|        98317.255|           2|
|           S-Type|99692.82|92520.76|192213.58000000002|96106.79000000001|           2|
|              Fit|99627.45|55448.35|          155075.8|          77537.9|           2|
|            Regal|99624.12|78089.06|         369555.66|        92388.915|           4|
|               62|99574.51|99574.51|          99574.51|         99574.51|           1|
|            Space|99528.78|9952

#Utilizando PYSPARK


In [55]:
#AGREGANDO OS DADOS
df_carros_spark = df_carros.agg(
     max("preco").alias("MAIOR"),
     min("preco").alias("MENOR"),
     sum("preco").alias("SOMA"),
     avg("preco").alias("MEDIA"),
     count("*").alias("TOTAL_CARROS")
)
df_carros_spark.show()

+--------+--------+-------------------+-----------------+------------+
|   MAIOR|   MENOR|               SOMA|            MEDIA|TOTAL_CARROS|
+--------+--------+-------------------+-----------------+------------+
|99938.89|45082.42|7.266464454000008E7|72664.64454000008|        1000|
+--------+--------+-------------------+-----------------+------------+



In [59]:
#AGRUPANDO OS DADOS COM GROUPBY
df_carros_spark = df_carros.groupBy("modelo_carro"
).agg(
     max("preco").alias("MAIOR"),
     min("preco").alias("MENOR"),
     sum("preco").alias("SOMA"),
     avg("preco").alias("MEDIA"),
     count("*").alias("TOTAL_CARROS")
)
df_carros_spark.show()

+------------+--------+--------+------------------+------------------+------------+
|modelo_carro|   MAIOR|   MENOR|              SOMA|             MEDIA|TOTAL_CARROS|
+------------+--------+--------+------------------+------------------+------------+
| Savana 1500|85392.12|49351.82|         324443.83|         64888.766|           5|
|      Cirrus|81784.17|81784.17|          81784.17|          81784.17|           1|
|         944|87930.76|87930.76|          87930.76|          87930.76|           1|
|      Avanti|98629.28|98629.28|          98629.28|          98629.28|           1|
|     Outlook|68318.15|49510.92|117829.06999999999|58914.534999999996|           2|
|     Contour|72937.19|72937.19|          72937.19|          72937.19|           1|
|         S60|85793.33| 48719.1|         377417.08| 62902.84666666667|           6|
|        1500|70293.96|45539.22|115833.18000000001|57916.590000000004|           2|
|       XLR-V|75537.75|50360.58|         125898.33|         62949.165|      

In [60]:
#Ordenando os dados com ORDER BY
df_carros_spark = df_carros.groupBy("modelo_carro"
).agg(
     max("preco").alias("MAIOR"),
     min("preco").alias("MENOR"),
     sum("preco").alias("SOMA"),
     avg("preco").alias("MEDIA"),
     count("*").alias("TOTAL_CARROS")
).orderBy(desc("MAIOR"))
df_carros_spark.show()

+-----------------+--------+--------+------------------+-----------------+------------+
|     modelo_carro|   MAIOR|   MENOR|              SOMA|            MEDIA|TOTAL_CARROS|
+-----------------+--------+--------+------------------+-----------------+------------+
|           Carens|99938.89|99938.89|          99938.89|         99938.89|           1|
|               X5|99866.05|93976.05|         289115.56|96371.85333333333|           3|
|     Express 2500|99803.08|56653.51|         156456.59|        78228.295|           2|
|           Sephia|99726.07|96908.44|         196634.51|        98317.255|           2|
|           S-Type|99692.82|92520.76|192213.58000000002|96106.79000000001|           2|
|              Fit|99627.45|55448.35|          155075.8|          77537.9|           2|
|            Regal|99624.12|78089.06|         369555.66|        92388.915|           4|
|               62|99574.51|99574.51|          99574.51|         99574.51|           1|
|            Space|99528.78|9952